# その他サンプリングの機能比較

scikit-learn の <b>resample関数</b>について、為念調査しました。

挙動としては単純にサンプルに揺れを作った感じです。

また、すべてのラベルに対してではないのですが、期待しているものと逆の挙動が見られました。

（すなわち、minority に対して under-sampling が行われ、majority に対してover-sampling されてしまう）

こちらも残念ながら、本件問題解決には使用できないと判断しております。

## (1) テストデータ／環境準備

マイオペで使用しているテストデータ（learning/tests/engine/fixtures/ 配下のCSVファイル）をベースに動作確認を行います。

動作確認にあたっては、MySQLdb に接続できないため、ローカル環境テスト用の Bot クラスを使用しています。

In [1]:
'''
    テスト環境を準備するためのモジュールを使用します。
'''
import sys
import os
learning_dir = os.path.abspath("../../") #<--- donusagi-bot/learning
os.chdir(learning_dir)

if learning_dir not in sys.path:
    sys.path.append(learning_dir)

from prototype.modules import TestTool

In [2]:
'''
    データファイルは、既存の訓練データを別場所にコピーしてから使用します
    テストデータは、csv_file_name で指定したものを使用します。
'''
csv_file_name = 'test_benefitone_conversation.csv'
copied_csv_file_path = TestTool.copy_testdata_csv(learning_dir, csv_file_name)

CSV file for test=[/Users/makmorit/GitHub/donusagi-bot/learning/prototype/resources/test_benefitone_conversation.csv]


## (2) TF-IDFベクターの準備

Bot クラス内に組み込まれている __build_training_set_from_csv 関数をバラして実行しています。

In [3]:
'''
    初期設定
    データファイル、エンコードを指定
    内容は、learn.py を参考にしました。    
'''
from learning.core.learn.learning_parameter import LearningParameter
attr = {
    'include_failed_data': False,
    'include_tag_vector': False,
    'classify_threshold': None,
    # 'algorithm': LearningParameter.ALGORITHM_NAIVE_BAYES
    'algorithm': LearningParameter.ALGORITHM_LOGISTIC_REGRESSION,
    # 'params_for_algorithm': { 'C': 200 }
    'params_for_algorithm': {}
}
learning_parameter = LearningParameter(attr)

bot_id = 7777
csv_file_path = copied_csv_file_path
csv_file_encoding = 'utf-8'

In [4]:
'''
    訓練データの生成（内部で TF-IDF 処理を実行）
'''
from learning.core.training_set.training_message_from_csv import TrainingMessageFromCsv
training_set = TrainingMessageFromCsv(bot_id, csv_file_path, learning_parameter, encoding=csv_file_encoding)
build_training_set_from_csv = training_set.build()

2017/03/15 PM 03:24:09 TrainingMessageFromCsv#__build_learning_training_messages count of learning data: 4114
2017/03/15 PM 03:24:09 TextArray#__init__ start
2017/03/15 PM 03:24:11 TextArray#to_vec start
2017/03/15 PM 03:24:11 TextArray#to_vec end


In [5]:
'''
    ラベルごとのサンプル数を調査します。
'''
X = build_training_set_from_csv.x
y = build_training_set_from_csv.y
count = TestTool.count_sample_by_label(y)
count

[[4677, 14],
 [4678, 2584],
 [4679, 14],
 [4680, 14],
 [4683, 14],
 [4686, 14],
 [4687, 8],
 [4690, 8],
 [4691, 14],
 [4692, 44],
 [4693, 30],
 [4700, 12],
 [4707, 18],
 [4708, 14],
 [4709, 14],
 [4710, 14],
 [4711, 14],
 [4712, 16],
 [4713, 14],
 [4718, 20],
 [4719, 8],
 [4720, 20],
 [4721, 8],
 [4724, 8],
 [4727, 14],
 [4728, 62],
 [4729, 26],
 [4730, 20],
 [4731, 14],
 [4732, 22],
 [4733, 14],
 [4734, 34],
 [4735, 12],
 [4738, 14],
 [4739, 14],
 [4740, 14],
 [4741, 8],
 [4742, 20],
 [4743, 14],
 [4744, 14],
 [4745, 20],
 [4750, 22],
 [4751, 20],
 [4752, 14],
 [4753, 8],
 [4754, 8],
 [4755, 14],
 [4756, 20],
 [4757, 8],
 [4758, 28],
 [4759, 14],
 [4760, 8],
 [4761, 20],
 [4762, 8],
 [4766, 14],
 [4767, 14],
 [4772, 60],
 [4776, 8],
 [4781, 14],
 [4782, 14],
 [4783, 20],
 [4786, 8],
 [4792, 20],
 [4793, 8],
 [4794, 8],
 [4795, 14],
 [4797, 42],
 [4798, 36],
 [4800, 14],
 [4802, 14],
 [4804, 14],
 [4807, 8],
 [4808, 106],
 [4817, 26],
 [4821, 24],
 [4824, 16],
 [4827, 8],
 [4830, 22],


## (3) resample関数の実行

In [6]:
'''
    utils.resample with default
'''
from sklearn import utils
resampled_X, resampled_y = utils.resample(X, y, random_state=0)

In [7]:
resampled_count = TestTool.count_sample_by_label(resampled_y)
resampled_count

[[4677, 9],
 [4678, 2603],
 [4679, 14],
 [4680, 12],
 [4683, 10],
 [4686, 16],
 [4687, 7],
 [4690, 6],
 [4691, 15],
 [4692, 50],
 [4693, 26],
 [4700, 11],
 [4707, 12],
 [4708, 13],
 [4709, 9],
 [4710, 15],
 [4711, 16],
 [4712, 15],
 [4713, 14],
 [4718, 30],
 [4719, 5],
 [4720, 26],
 [4721, 9],
 [4724, 10],
 [4727, 11],
 [4728, 52],
 [4729, 14],
 [4730, 29],
 [4731, 14],
 [4732, 22],
 [4733, 9],
 [4734, 35],
 [4735, 10],
 [4738, 16],
 [4739, 18],
 [4740, 17],
 [4741, 5],
 [4742, 23],
 [4743, 16],
 [4744, 19],
 [4745, 23],
 [4750, 27],
 [4751, 18],
 [4752, 15],
 [4753, 8],
 [4754, 9],
 [4755, 10],
 [4756, 15],
 [4757, 6],
 [4758, 26],
 [4759, 16],
 [4760, 5],
 [4761, 17],
 [4762, 8],
 [4766, 17],
 [4767, 16],
 [4772, 58],
 [4776, 13],
 [4781, 17],
 [4782, 9],
 [4783, 24],
 [4786, 4],
 [4792, 20],
 [4793, 11],
 [4794, 7],
 [4795, 16],
 [4797, 43],
 [4798, 41],
 [4800, 15],
 [4802, 9],
 [4804, 12],
 [4807, 7],
 [4808, 97],
 [4817, 16],
 [4821, 23],
 [4824, 12],
 [4827, 9],
 [4830, 20],
 [4